In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
;; Use full paths for ACL2-MCP compatibility
(include-book "/workspaces/acl2-swf-experiments/experiments/agents/verified-agent")
(include-book "/workspaces/acl2-swf-experiments/experiments/agents/llm-client")

Note (from clause-processors/equality): disabling DISJOIN, DISJOIN2,
CONJOIN and CONJOIN2.


Summary
Form:  ( INCLUDE-BOOK 
"/workspaces/acl2-swf-experiments/experiments/agents/verified-agent"
...)
Rules: NIL
Time:  0.84 seconds (prove: 0.00, print: 0.00, other: 0.84)
 "/workspaces/acl2-swf-experiments/experiments/agents/verified-agent.lisp"

TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK 
"/workspaces/acl2-swf-experiments/experiments/agents/llm-client" ...):
The ttag note just printed to the terminal indicates a modification
to ACL2.  To avoid this warning, supply an explicit :TTAGS argument
when including the book 
"/workspaces/acl2-swf-experiments/experiments/agents/llm-client".


TTAG NOTE (for included book): Adding ttag :QUICKLISP.DEXADOR from book /home/acl2/books/quicklisp/dexador.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK 
"/workspaces/acl2-swf-experiments/experiments/agents/llm-client" ...

In [3]:
;; Model preferences in order - first match wins
(defconst *model-prefs* '("qwen" "nemotron" "llama" "gemma"))


Summary
Form:  ( DEFCONST *MODEL-PREFS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MODEL-PREFS*


In [4]:
;; Get full model info and select best loaded completions model
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (prog2$ 
    (if err
        (cw "~%Error getting models: ~s0~%" err)
      (progn$
       (cw "~%Available models: ~x0~%" (len models))
       (cw "Loaded completions models: ~x0~%" 
           (len (filter-loaded-completions-models models)))))
    (mv nil '(value-triple :models-listed) state))))


Available models: 14
Loaded completions models: 3

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 :MODELS-LISTED


In [5]:
;; Select the best model based on preferences
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable err))
   (let ((selected (select-completions-model models *model-prefs*)))
     (prog2$
      (if selected
          (cw "~%Selected model: ~s0 (context: ~x1 tokens)~%"
              (model-info->id selected)
              (model-info->loaded-context-length selected))
        (cw "~%WARNING: No suitable model found! Make sure LM Studio is running.~%"))
      (mv nil 
          `(defconst *model-id* 
             ,(if selected (model-info->id selected) "no-model-found"))
          state)))))


Selected model: qwen/qwen3-coder-30b (context: 80000 tokens)

Summary
Form:  ( DEFCONST *MODEL-ID* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MODEL-ID*


In [6]:
(defconst *initial-state*
  (make-agent-state 
    :max-steps 20              ; Allow up to 20 conversation turns
    :token-budget 50000        ; Token budget for tool calls
    :time-budget 3600          ; 1 hour time budget
    :file-access 1             ; Read access to files
    :execute-allowed nil       ; No code execution
    :max-context-tokens 8000   ; Context window size
    :satisfaction 0))          ; Starting satisfaction


Summary
Form:  ( DEFCONST *INITIAL-STATE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *INITIAL-STATE*


In [7]:
(defconst *system-prompt*
  "You are a helpful AI assistant running inside a formally verified agent framework built in ACL2. 
You can engage in conversation and help with questions.
Be concise but helpful in your responses.")


Summary
Form:  ( DEFCONST *SYSTEM-PROMPT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SYSTEM-PROMPT*


In [8]:
(defconst *agent-v1*
  (init-agent-conversation *system-prompt* *initial-state*))


Summary
Form:  ( DEFCONST *AGENT-V1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V1*


In [9]:
;; Verify the conversation was initialized
(defconst *check-init*
  (prog2$ (cw "Agent initialized. Messages: ~x0~%" 
              (len (get-messages *agent-v1*)))
          t))

Agent initialized. Messages: 1

Summary
Form:  ( DEFCONST *CHECK-INIT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHECK-INIT*


In [10]:
(defun show-messages (msgs)
  (declare (xargs :mode :program))
  (if (endp msgs)
      nil
    (let* ((msg (car msgs))
           (role (chat-message->role msg))
           (content (chat-message->content msg))
           (role-str (chat-role-case role
                       :system "SYSTEM"
                       :user "USER"
                       :assistant "ASSISTANT"
                       :tool "TOOL")))
      (prog2$ (cw "~%[~s0]~%~s1~%" role-str content)
              (show-messages (cdr msgs))))))


Summary
Form:  ( DEFUN SHOW-MESSAGES ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 SHOW-MESSAGES


In [11]:
(defun show-conversation (st)
  (declare (xargs :mode :program))
  (prog2$ (cw "~%========== Conversation ==========")
          (prog2$ (show-messages (get-messages st))
                  (cw "~%===================================~%"))))


Summary
Form:  ( DEFUN SHOW-CONVERSATION ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SHOW-CONVERSATION


In [12]:
(defconst *agent-v2*
  (add-user-msg "Hello! Can you explain what a formally verified agent is?" 
                *agent-v1*))


Summary
Form:  ( DEFCONST *AGENT-V2* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V2*


In [13]:
;; Show the conversation so far
(defconst *show-v2*
  (show-conversation *agent-v2*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?


Summary
Form:  ( DEFCONST *SHOW-V2* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V2*


In [14]:
;; Call LLM and print the response
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *agent-v2*) state)
   (prog2$ 
    (if err
        (cw "~%LLM Error: ~s0~%" err)
      (cw "~%LLM Response:~%~s0~%" response))
    (mv nil '(value-triple :llm-called) state))))


LLM Response:
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
safety, correctness, or security properties within a formal framework.

In the context of ACL2 (A Computational Logic for Applicative Common
Lisp), this means:

1. **Formal specification**: The agent's behavior is precisely defined
using mathematical logic
2. **Mathematical proof**: Properties like safety constraints, correctness
conditions, or security requirements are proven using automated theorem
proving
3. **Verification framework**: ACL2 provides the logical foundation
and proof infrastructure to establish these properties

Key benefits include:
- **Guaranteed properties**: The agent will always behave according
to its formal specification
- **Bug prevention**: Critical errors that violate safety constraints
are mathematically impossible
- **Trustworthiness**: High confidence in the agent's behavior, especially
important for sa

In [15]:
(defconst *assistant-response-1*
  "A formally verified agent is an AI system whose decision-making logic has been mathematically proven correct using a theorem prover like ACL2. This means properties like 'the agent will always respect permission boundaries' or 'the agent will eventually terminate' are guaranteed by proof, not just testing. The agent you're talking to right now has verified properties for permission safety, budget bounds, and termination!")


Summary
Form:  ( DEFCONST *ASSISTANT-RESPONSE-1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *ASSISTANT-RESPONSE-1*


In [16]:
(defconst *agent-v3*
  (add-assistant-msg *assistant-response-1* *agent-v2*))


Summary
Form:  ( DEFCONST *AGENT-V3* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V3*


In [17]:
;; Show updated conversation
(defconst *show-v3*
  (show-conversation *agent-v3*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose decision-making logic
has been mathematically proven correct using a theorem prover like
ACL2. This means properties like 'the agent will always respect permission
boundaries' or 'the agent will eventually terminate' are guaranteed
by proof, not just testing. The agent you're talking to right now has
verified properties for permission safety, budget bounds, and termination!


Summary
Form:  ( DEFCONST *SHOW-V3* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V3*


In [18]:
(defconst *state-check-1*
  (prog2$ (cw "~%Agent State Check:~%")
    (prog2$ (cw "  agent-state-p: ~x0~%" (agent-state-p *agent-v3*))
      (prog2$ (cw "  must-respond-p: ~x0~%" (must-respond-p *agent-v3*))
        (prog2$ (cw "  should-continue-p: ~x0~%" (should-continue-p *agent-v3*))
          (prog2$ (cw "  step-counter: ~x0~%" (agent-state->step-counter *agent-v3*))
            (prog2$ (cw "  messages count: ~x0~%" (len (get-messages *agent-v3*)))
              (cw "  conversation-has-room-p: ~x0~%" 
                  (conversation-has-room-p *agent-v3*)))))))))


Agent State Check:
  agent-state-p: T
  must-respond-p: NIL
  should-continue-p: T
  step-counter: 0
  messages count: 3
  conversation-has-room-p: T

Summary
Form:  ( DEFCONST *STATE-CHECK-1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *STATE-CHECK-1*


In [19]:
(defconst *agent-v4*
  (add-user-msg "What properties have been proven about this agent?" 
                *agent-v3*))


Summary
Form:  ( DEFCONST *AGENT-V4* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V4*


In [20]:
(defconst *show-v4*
  (show-conversation *agent-v4*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose decision-making logic
has been mathematically proven correct using a theorem prover like
ACL2. This means properties like 'the agent will always respect permission
boundaries' or 'the agent will eventually terminate' are guaranteed
by proof, not just testing. The agent you're talking to right now has
verified properties for permission safety, budget bounds, and termination!

[USER]
What properties have been proven about this agent?


Summary
Form:  ( DEFCONST *SHOW-V4* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V4*


In [35]:
(mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *agent-v4*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv "Error" state))
     (prog2$ (cw "~%LLM Response:~%~s0~%" response)
             (mv response state))))


LLM Response:
Based on the ACL2 framework I'm running in, I can tell you that my
properties have been formally verified to include:

1. **Permission safety** - I won't violate permission boundaries or
access unauthorized resources
2. **Budget bounds** - My computational resources (time, memory) are
bounded and won't exceed specified limits  
3. **Termination** - I will always terminate after a finite number
of steps
4. **Output correctness** - My responses are guaranteed to be consistent
with the system's specification

These properties are proven using ACL2's theorem proving capabilities,
so they're mathematically guaranteed rather than just empirically tested.
This means you can trust that I'll behave within these verified constraints
in all scenarios.

The formal verification ensures that even in edge cases or unexpected
inputs, these safety properties will hold.
("Based on the ACL2 framework I'm running in, I can tell you that my properties have been formally verified to include:


In [ ]:
(defconst *assistant-response-2*
  "Several key properties have been formally proven:

1. **Permission Safety** - The agent can only invoke tools it has permission for
2. **Budget Bounds** - Token and time budgets never go negative after deductions
3. **Termination** - The agent is guaranteed to stop (max steps, budget exhaustion, or completion)
4. **State Preservation** - All state transitions preserve the validity of agent state
5. **Conversation Preservation** - Adding messages doesn't affect control flow decisions

These aren't just tested - they're mathematically proven to hold for ALL possible inputs!")

In [22]:
(defconst *agent-v5*
  (add-assistant-msg *assistant-response-2* *agent-v4*))


Summary
Form:  ( DEFCONST *AGENT-V5* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V5*


In [23]:
(defconst *show-v5*
  (show-conversation *agent-v5*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose decision-making logic
has been mathematically proven correct using a theorem prover like
ACL2. This means properties like 'the agent will always respect permission
boundaries' or 'the agent will eventually terminate' are guaranteed
by proof, not just testing. The agent you're talking to right now has
verified properties for permission safety, budget bounds, and termination!

[USER]
What properties have been proven about this agent?

[ASSISTANT]
Several key properties have been formally proven:  1. **Permission
Safety** - The agent can only invoke tools it has permission for 2.
**Budget Bounds** - Token and time budgets never

In [24]:
(defconst *agent-v6*
  (add-user-msg "That's impressive! How does the context management work?" 
                *agent-v5*))


Summary
Form:  ( DEFCONST *AGENT-V6* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V6*


In [25]:
(defconst *show-v6*
  (show-conversation *agent-v6*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose decision-making logic
has been mathematically proven correct using a theorem prover like
ACL2. This means properties like 'the agent will always respect permission
boundaries' or 'the agent will eventually terminate' are guaranteed
by proof, not just testing. The agent you're talking to right now has
verified properties for permission safety, budget bounds, and termination!

[USER]
What properties have been proven about this agent?

[ASSISTANT]
Several key properties have been formally proven:  1. **Permission
Safety** - The agent can only invoke tools it has permission for 2.
**Budget Bounds** - Token and time budgets never

In [26]:
(defconst *context-check*
  (let* ((msgs (get-messages *agent-v6*))
         (char-len (messages-char-length msgs))
         (token-est (messages-token-estimate msgs))
         (max-tokens (agent-state->max-context-tokens *agent-v6*)))
    (prog2$ (cw "~%Context Usage:~%")
      (prog2$ (cw "  Total characters: ~x0~%" char-len)
        (prog2$ (cw "  Estimated tokens: ~x0~%" token-est)
          (prog2$ (cw "  Max tokens: ~x0~%" max-tokens)
            (prog2$ (cw "  Safety margin: ~x0~%" *context-safety-margin*)
              (cw "  Fits in context: ~x0~%" 
                  (messages-fit-p msgs max-tokens)))))))))


Context Usage:
  Total characters: 1360
  Estimated tokens: 340
  Max tokens: 8000
  Safety margin: 500
  Fits in context: T

Summary
Form:  ( DEFCONST *CONTEXT-CHECK* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CONTEXT-CHECK*


In [27]:
;; Simulate calling a file-read tool
(defconst *agent-with-tool*
  (add-tool-result 
    "File contents of verified-agent.lisp (first 500 chars):
; Verified ReAct Agent - ACL2 Implementation
; This file implements a formally verified ReAct agent...
(in-package \"ACL2\")
(include-book \"std/util/define\" :dir :system)
..." 
    *agent-v6*))


Summary
Form:  ( DEFCONST *AGENT-WITH-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-WITH-TOOL*


In [28]:
(defconst *show-tool*
  (show-conversation *agent-with-tool*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose decision-making logic
has been mathematically proven correct using a theorem prover like
ACL2. This means properties like 'the agent will always respect permission
boundaries' or 'the agent will eventually terminate' are guaranteed
by proof, not just testing. The agent you're talking to right now has
verified properties for permission safety, budget bounds, and termination!

[USER]
What properties have been proven about this agent?

[ASSISTANT]
Several key properties have been formally proven:  1. **Permission
Safety** - The agent can only invoke tools it has permission for 2.
**Budget Bounds** - Token and time budgets never

In [29]:
(defun chat-turn (user-msg agent-st model-id state)
  "Execute one chat turn: add user message, call LLM, add response"
  (declare (xargs :mode :program :stobjs state))
  (let ((st-with-user (add-user-msg user-msg agent-st)))
    (mv-let (err response state)
      (llm-chat-completion model-id (get-messages st-with-user) state)
      (if err
          (prog2$ (cw "~%Error: ~s0~%" err)
                  (mv st-with-user state))
        (let ((st-with-response (add-assistant-msg response st-with-user)))
          (prog2$ (cw "~%Assistant: ~s0~%" response)
                  (mv st-with-response state)))))))


Summary
Form:  ( DEFUN CHAT-TURN ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHAT-TURN


In [30]:
;; Create a tool spec for demonstration
(defconst *read-file-tool*
  (make-tool-spec
    :name 'read-file
    :required-access 1      ; Needs read access
    :requires-execute nil
    :token-cost 100
    :time-cost 5))


Summary
Form:  ( DEFCONST *READ-FILE-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *READ-FILE-TOOL*


In [31]:
;; Check if we can invoke this tool
(defconst *can-read-file*
  (prog2$ (cw "~%Can invoke read-file tool: ~x0~%" 
              (can-invoke-tool-p *read-file-tool* *agent-v6*))
          (can-invoke-tool-p *read-file-tool* *agent-v6*)))


Can invoke read-file tool: T

Summary
Form:  ( DEFCONST *CAN-READ-FILE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CAN-READ-FILE*


In [32]:
;; Simulate a ReAct step with conversation
(defconst *agent-after-react*
  (if (and (not (must-respond-p *agent-v6*))
           (can-invoke-tool-p *read-file-tool* *agent-v6*))
      (react-step-with-conversation
        (agent-action-tool-call 'read-file "verified-agent.lisp")
        *read-file-tool*
        "I'll read the verified-agent.lisp file to show you the implementation."
        "File read successfully. Contents: (in-package ACL2)..."
        *agent-v6*)
    *agent-v6*))


Summary
Form:  ( DEFCONST *AGENT-AFTER-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-AFTER-REACT*


In [33]:
(defconst *show-react*
  (prog2$ (cw "~%After ReAct step:~%")
    (prog2$ (cw "  Step counter: ~x0~%" 
                (agent-state->step-counter *agent-after-react*))
      (prog2$ (cw "  Token budget: ~x0~%" 
                  (agent-state->token-budget *agent-after-react*))
        (cw "  Message count: ~x0~%" 
            (len (get-messages *agent-after-react*)))))))


After ReAct step:
  Step counter: 1
  Token budget: 49900
  Message count: 8

Summary
Form:  ( DEFCONST *SHOW-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-REACT*


In [34]:
(defconst *demo-complete*
  (prog2$ (cw "~%~%========================================~%")
    (prog2$ (cw "Chat Demo Complete!~%")
      (prog2$ (cw "========================================~%")
        (prog2$ (cw "~%Key points demonstrated:~%")
          (prog2$ (cw "  1. Agent state initialization~%")
            (prog2$ (cw "  2. Conversation management (add-user-msg, add-assistant-msg)~%")
              (prog2$ (cw "  3. Context length tracking~%")
                (prog2$ (cw "  4. Tool result handling~%")
                  (prog2$ (cw "  5. ReAct step with conversation~%")
                    (prog2$ (cw "  6. State preservation verification~%")
                      (cw "~%For live LLM chat, ensure LM Studio is running at:~%  http://host.docker.internal:1234~%"))))))))))))



Chat Demo Complete!

Key points demonstrated:
  1. Agent state initialization
  2. Conversation management (add-user-msg, add-assistant-msg)
  3. Context length tracking
  4. Tool result handling
  5. ReAct step with conversation
  6. State preservation verification

For live LLM chat, ensure LM Studio is running at:
  http://host.docker.internal:1234

Summary
Form:  ( DEFCONST *DEMO-COMPLETE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-COMPLETE*
